In [ ]:
import panel as pn
import pandas as pd
import holoviews as hv

from ipyleaflet import (
    Map,
    Rectangle,
    LayersControl,
    LayerGroup,
    Marker,
    WMSLayer,
)

pn.extension()
pn.extension("ipywidgets")
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import concurrent
from dataclasses import dataclass, field

import ephem
import numpy as np
import prosail

pd.options.plotting.backend = "holoviews"


In [ ]:
@dataclass
class ThermalSail:
    """A data class to store thermal SAIL parameters.
    Note that the view/illumination geometry isn't stored
    here.
    """

    lam: float
    lai: float
    tsoil: float
    tleaf: float
    emv: float
    ems: float
    hspot: float
    lidfa: float
    tatm: float = field(default=-14)
    lidfb: float = field(default=None)
    tleaf_sunlit: float = field(default=None)
    tsoil_sunlit: float = field(default=None)

    def __post_init__(self):
        if self.tleaf_sunlit is None:
            self.tleaf_sunlit = self.tleaf
        if self.tsoil_sunlit is None:
            self.tsoil_sunlit = self.tsoil
        """_summary_
        """


def calculate_solar_position(
    longitude: float, latitude: float, time: dt.datetime | str
) -> tuple:
    """Calculate solar position for a given location and time.
    Longitude and latitude are in decimal degrees. Time can either
    be a string or a datetime object.


    Args:
        longitude (float): longitude in decimal degrees
        latitude (float): latitude in decimal degrees
        time (Union): time, either as datetime or string

    Returns:
        tuple: sza and saa in degrees
    """
    lat = str(latitude)
    lon = str(longitude)
    loc = ephem.Observer()
    loc.lon = lon
    loc.lat = lat
    if type(time) == str:
        loc.date = time
    else:
        loc.date = time.strftime("%Y/%m/%d %H:%M")
    sun = ephem.Sun(loc)
    zenith = 90.0 - np.rad2deg(sun.alt)
    azimuth = np.rad2deg(sun.az)
    return zenith, azimuth


def calculate_bean_extinction(theta, lad):
    """Calculate bean extinction coefficient kappa
    according to Campbell & Norman

    Args:
        theta (float): View zenith angle (degrees)
        lad (float): Leaf angle distribution parameter. 1 for spherical

    Returns:
        float
    """
    theta = np.radians(theta)
    return np.sqrt(lad**2 + np.tan(theta) ** 2) / (
        lad + 1.774 * (lad + 1.182) ** -0.733
    )


def calculate_clumping(H, w, D, L, lad, theta, psi):
    """Calculates clumping parameter

    Args:
        H (float): Row height [m]
        w (float): Row width [m]
        D (float): Row separation [m]
        L (float): Leaf area index [m2/m2]
        lad (float): Leaf angle distribution  parameter. 1 for spherical
        theta (float): VZA (deg)
        psi (float): Relative azimuth angle between view and row orientation (deg)
    """
    # kappa (beam extinction)
    kappa_be = calculate_bean_extinction(theta, lad)
    # Apparent shaded fraction
    f = (
        w + H * np.tan(np.radians(theta)) * np.abs(np.sin(np.radians(psi)))
    ) / D
    f = np.minimum(1.0, f)  # Clip to f to be <=1
    # Canopy grap fraction
    gap_fraction = f * np.exp(-kappa_be * L) + (1.0 - f)
    return -np.log(gap_fraction) / (kappa_be * L)


def thermal_polar_sims(sza, parameters, vzas=None, raas=None):
    """Create a thermal polar plot for a given solar zenith angle,
    a SAIL parameter instance

    Args:
        sza (float): _description_
        parameters (ThermalSail): _description_
        vzas (iterable, optional): _description_. Defaults to np.linspace(0,90,5).
        raas (iterable, optional): _description_. Defaults to np.linspace(0,180, 30).

    Returns:
        _type_: _description_
    """
    if vzas is None:
        vzas = np.arange(0, 90, 5)
    if raas is None:
        raas = np.arange(0, 190, 10)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        index_futures = {}
        i = 0
        angles = []
        for vza in vzas:
            for raa in raas:
                future = executor.submit(
                    run_thermalsail, sza, vza, raa, parameters
                )
                futures.append(future)
                index_futures[future] = i
                angles.append([sza, vza, raa])
                i += 1

        results = [None] * len(futures)
        for future in concurrent.futures.as_completed(futures):
            index = index_futures[future]
            result = future.result()
            results[index] = result[1] - 273.15

        return angles, results


def run_thermalsail(sza, vza, raa, parameters):
    return run_thermal_sail(
        sza,
        vza,
        raa,
        parameters.lam,
        parameters.tsoil,
        parameters.tleaf,
        parameters.lai,
        parameters.lidfa,
        parameters.lidfb,
        parameters.hspot,
        parameters.emv,
        parameters.ems,
        tveg_sunlit=parameters.tleaf_sunlit,
        tsoil_sunlit=parameters.tsoil_sunlit,
        t_atm=parameters.tatm,
    )


def run_thermal_sail(
    sza,
    vza,
    raa,
    lam,
    tsoil,
    tveg,
    lai,
    lidfa,
    lidfb,
    hspot,
    emv,
    ems,
    tveg_sunlit=None,
    tsoil_sunlit=None,
    t_atm=-14,
):
    """Run SAIl simulations for a vegetation row scene. Basically, we modify LAI to be
    effective LAI, and calculate the clumping parameter in terms of having the equivalent
    grap fraction for a discontinuous canopy arranged in rectangular rows and a continuous
    equivalent canopy."""

    # Only two components
    if tveg_sunlit is None:
        tveg_sunlit = tveg
    if tsoil_sunlit is None:
        tsoil_sunlit = tsoil
    if lidfb is None:
        typelidf = 2
        lidfb = 0.0
    else:
        typelidf = 1
    return prosail.run_thermal_sail(
        lam,
        tveg + 273.15,
        tsoil + 273.15,
        tveg_sunlit + 273.15,
        tsoil_sunlit + 273.15,
        t_atm + 273.15,
        lai,
        lidfa,
        hspot,
        sza,
        vza,
        raa,
        emv=emv,
        ems=ems,
        lidfb=lidfb,
        typelidf=typelidf,
    )


In [ ]:
def do_sims(
    Lon,
    Lat,
    Time,
    lam,
    lai,
    tsoil,
    tleaf,
    emv,
    ems,
    hspot,
    lidfa,
    tleaf_sunlit,
    tsoil_sunlit,
    H,
    w,
    D,
    psi,
):
    sza, saa = calculate_solar_position(Lon, Lat, Time)
    sail_params = ThermalSail(
        lam=lam,
        lai=lai,
        tsoil=tsoil,
        tleaf=tleaf,
        emv=emv,
        ems=ems,
        hspot=hspot,
        lidfa=lidfa,
        lidfb=None,
        tleaf_sunlit=tleaf_sunlit,
        tsoil_sunlit=tsoil_sunlit,
    )
    vza = np.linspace(-90, 90, 90)
    raa = [0 if v <= 0 else 180 for v in vza]
    retval = [
        run_thermalsail(sza, abs(v), r, sail_params) for v, r in zip(vza, raa)
    ]
    retval = np.array([x[1] - 273.15 for x in retval])
    scatter = hv.Scatter(
        (vza, retval), "VZA [deg]", "Brightness temperature [degC]"
    ).opts(width=800)
    curve = hv.Curve(scatter, label="Continuous canopy")
    retval = curve * scatter
    p = ThermalSail(**sail_params.__dict__.copy())
    row_sims = []
    for v, r in zip(vza, raa):
        Omega = calculate_clumping(H, w, D, lai, sail_params.lidfa, v, psi)
        p.lai = lai * np.abs(Omega)
        x = run_thermalsail(sza, abs(v), r, p)
        row_sims.append(x[1] - 273.15)
    scatter = hv.Scatter(
        (vza, row_sims), "VZA [deg]", "Brightness temperature [degC]"
    ).opts(width=800)
    curve = hv.Curve(scatter, label="Row-oriented canopy")

    return retval * scatter * curve


# retval = do_sims()


In [ ]:
# Acquisition
lon = pn.widgets.FloatInput(
    name="Longitude", start=-180, end=180, value=11.124
)
lat = pn.widgets.FloatInput(name="Latitude", start=-90, end=90, value=42.7635)
timer = pn.widgets.DatetimePicker(
    name="Overflight time", value=dt.datetime(2023, 5, 18, 14, 0)
)
lam = pn.widgets.FloatSlider(
    name="Wavelength [um]",
    start=7.6,
    end=12.5,
    step=(12.5 - 7.6) / 103,
    value=9.5,
)
acquisition = pn.WidgetBox("### Acquisition", lon, lat, timer, lam)

# Architecture
hspot = pn.widgets.FloatSlider(
    name="HotSpot parameter [-]", start=0.01, end=0.1, value=0.05
)
lidfa = pn.widgets.FloatSlider(
    name="Average leaf angle [deg]", start=0.0, end=90, value=45
)
lai = pn.widgets.FloatSlider(
    name="Leaf Area Index [m2/m2]", start=0.0, end=4, value=2
)
architecture = pn.WidgetBox("### Canopy", lai, lidfa, hspot)
# Thermodynamics
tsoil = pn.widgets.FloatSlider(
    name="Shaded soil temperature [degC]", start=-10, end=100, value=35
)
tleaf = pn.widgets.FloatSlider(
    name="Shaded leaf temperature [degC]", start=-10, end=100, value=25
)
tsoil_sunlit = pn.widgets.FloatSlider(
    name="Sunlit soil temperature [degC]", start=-10, end=100, value=45
)
tleaf_sunlit = pn.widgets.FloatSlider(
    name="Sunlit leaf temperature [degC]", start=-10, end=100, value=33
)
ems = pn.widgets.FloatSlider(
    name="Soil emissivity [-]", start=0.9, end=1.0, value=0.94
)
emv = pn.widgets.FloatSlider(
    name="Leaf emissivity [-]", start=0.9, end=1.0, value=0.98
)
thermodynamics = pn.WidgetBox(
    "### Thermal", tsoil, tleaf, tsoil_sunlit, tleaf_sunlit, ems, emv
)


row_H = pn.widgets.FloatSlider(
    name="Row height [m]", start=0.0, end=5.0, value=1.5
)
row_width = pn.widgets.FloatSlider(
    name="Row width [m]", start=0.0, end=5.0, value=0.5
)
row_sep = pn.widgets.FloatSlider(
    name="Row separation [m]", start=0.0, end=10.0, value=1.2
)
row_angle = pn.widgets.FloatSlider(
    name="Row angle with VAA [deg]", start=0, end=90.0, value=45
)
rows = pn.WidgetBox("### Row geometry", row_H, row_width, row_sep, row_angle)


def handle_interaction(**kwargs):
    if kwargs.get("event") == "move":
        lon.value = kwargs["location"][1]
        lat.value = kwargs["location"][0]


m = Map(center=(42.7465, 11.1124), zoom_snap=0.25, zoom=10)
wms = WMSLayer(
    url="https://tiles.maps.eox.at/wms",
    layers="s2cloudless-2020_3857",
    format="image/jpeg",
    transparent=True,
    attribution="Sentinel2 Cloudless 2020",
)

marker = Marker(location=(42.7465, 11.1124), draggable=True)
m.add_layer(marker)
m.add_layer(wms)
marker.on_move(handle_interaction)
m.add_control(LayersControl(position="topright"))


interactive_simulations = pn.bind(
    do_sims,
    lon,
    lat,
    timer,
    lam,
    lai,
    tsoil,
    tleaf,
    emv,
    ems,
    hspot,
    lidfa,
    tleaf_sunlit,
    tsoil_sunlit,
    row_H,
    row_width,
    row_sep,
    row_angle,
)


template = pn.template.FastListTemplate(
    site="NCEO Airborne Facility",
    title="Brightness temperature simulations",
    sidebar=pn.Row(acquisition, architecture, thermodynamics, rows),
    main=pn.Column(
        m,
        pn.Row(
            pn.layout.HSpacer(),
            interactive_simulations,
            pn.layout.HSpacer(),
            width=800,
            height=600,
        ),
    ),
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
).servable()
template.show()


/home/ucfajlg/python/fenix_inversion/clumping.py:110: RuntimeWarning: divide by zero encountered in log
  return -np.log(gap_fraction) / (kappa_be * L)
/home/ucfajlg/python/prosail/prosail/FourSAIL.py:591: RuntimeWarning: invalid value encountered in double_scalars
  rsos = w * lai * sumint
/home/ucfajlg/python/prosail/prosail/FourSAIL.py:592: RuntimeWarning: invalid value encountered in double_scalars
  gammasos = ko * lai * sumint


Launching server at http://localhost:45689


<panel.io.server.Server>